In [1]:
from pathlib import Path
DATA_ROOT = Path("../data/jigsaw")

In [ ]:
# vocab should have been saved using
"""vocab.save_to_files(DATA_ROOT / "vocab")"""

In [2]:
from allennlp.data.vocabulary import Vocabulary
vocab = Vocabulary.from_files(DATA_ROOT / "vocab")

02/03/2019 08:08:18 - INFO - allennlp.data.vocabulary -   Loading token dictionary from ../data/jigsaw/vocab.


In [4]:
import fastText
ft_model = fastText.load_model(str(DATA_ROOT / "ft_model.bin"))

In [5]:
with (DATA_ROOT / "ft_model.txt").open("wt") as f:
    for idx, token in vocab.get_index_to_token_vocabulary().items():
        emb = ft_model.get_word_vector(token)
        emb_as_str = " ".join(["%.4f" % x for x in emb])
        f.write(f"{token} {emb_as_str}\n")